In [9]:
# parse .out files HI dtp ID IP LY MT ne OP SP IN HI-IN HI-IN_HI_dtp_OP HI-IN_HI_dtp_OP_NA
#! for register in HI-IN_HI_dtp; do python3 get_losses.py ../logs/parsed/${register}_register-1.71B-943* --tokens-per-step=2097152 > ./parsed_loss_files/${register}_losses_round1.tsv ; done

In [10]:
! pip -q install pandas plotly
! pip -q install nbformat>=4.2.0

In [11]:
! rm "=4.2.0"   # pip install version creates this *eyeroll*

In [12]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import glob

In [20]:
tsv_files = glob.glob("parsed_loss_files/*.tsv")


from collections import defaultdict

registers = ["HI","HI-IN","IN", "ID", "IP", "LY","MT","NA", "OP", "SP","ne", "dtp", "HI-IN_HI_dtp", "HI-IN_HI_dtp_OP", "HI-IN_HI_dtp_OP_NA"]

def regroup_files(array):
    # Create a dictionary to store groups
    grouped = defaultdict(list)

    # Iterate through the array and group by register
    for string in array:
        for register in registers:
            #print(string.split("/")[1].split("_losses")[0])
            if string.split("/")[1].split("_losses")[0] == register:
            #if string.split("/")[1].startswith(register):
                grouped[register].append(string)
                break  # Stop checking other registers once a match is found

    # Convert grouped dictionary values to a list of lists
    #result = list(grouped.values())
    result = [sorted(group) for group in grouped.values()]
    return result


#print(tsv_files)
ordered_files = regroup_files(tsv_files)
for p in ordered_files:
    print(p)



['parsed_loss_files/SP_losses_round1.tsv', 'parsed_loss_files/SP_losses_round2.tsv', 'parsed_loss_files/SP_losses_round3.tsv']
['parsed_loss_files/IN_losses_round2.tsv', 'parsed_loss_files/IN_losses_round3.tsv']
['parsed_loss_files/HI-IN_HI_dtp_OP_NA_losses_round1.tsv', 'parsed_loss_files/HI-IN_HI_dtp_OP_NA_losses_round2.tsv']
['parsed_loss_files/HI_losses_round1.tsv', 'parsed_loss_files/HI_losses_round2.tsv', 'parsed_loss_files/HI_losses_round3.tsv']
['parsed_loss_files/OP_losses_round1.tsv', 'parsed_loss_files/OP_losses_round2.tsv', 'parsed_loss_files/OP_losses_round3.tsv']
['parsed_loss_files/NA_losses_round1.tsv', 'parsed_loss_files/NA_losses_round2.tsv', 'parsed_loss_files/NA_losses_round3.tsv', 'parsed_loss_files/NA_losses_round4.tsv']
['parsed_loss_files/ne_losses_round1.tsv', 'parsed_loss_files/ne_losses_round2.tsv', 'parsed_loss_files/ne_losses_round3.tsv']
['parsed_loss_files/LY_losses_round1.tsv', 'parsed_loss_files/LY_losses_round2.tsv', 'parsed_loss_files/LY_losses_round3.

In [21]:


#epoch_sizes = {"HI":100e9, "ID":100e9, "IN":100e9, "IP":100e9, "NA":100e9, "MT":100e9, "OP":100e9, "ne":100e9, "dtp":100e9, "LY":20e9, "SP":40e9}
fig = go.Figure()
key="tokens"

for group in ordered_files:
    dfs = []
    for file in group:
        #print(f"now in {file}")
        df = pd.read_csv(file, sep='\t', skipfooter=3)
        df = df.apply(pd.to_numeric)
        dfs.append(df)
        register = file.split("/")[1].split("_losses")[0] #file.replace('.tsv', '')  # Add a 'Parameter' column with the filename as the value
    df = pd.concat(dfs)
    fig.add_trace(go.Scatter(
                x=df[key],
                y=df["loss"],
                mode='lines',
                name=f'{register}',
                legendgroup=f'{register}'
                )
            )
    #fig.add_vline(x=epoch_sizes[register], line_dash="dot", line_color="gray", annotation_text=f"epoch {register}", legendgroup=f'{register}')

fig.update_layout(
        title=f"Loss from first {len(ordered_files[0])} training runs per register",
        xaxis_title=key,
        yaxis_title="loss",
        legend_title="Legend",
        height=700
    )

#move HI annotation lower because it clashes with IP
#fig.update_layout(annotations=[{**a, **{"y":.97}} if "HI" in a["text"] else {**a} for a in fig.to_dict()["layout"]["annotations"] ])
    
fig.show()


/users/mynttiam/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.

